## Buoyancy driven flow 

Here we interpret buoyancy as having come from a thermal perturbation (not scaled)

**New concepts:** Different mesh types, initialising fields with numpy, a buoyancy _function_, 


In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import numpy as np

In [ ]:
res = 32
boxHeight = 1.0
aspect_ratio = 1.0

In [ ]:
# A mesh to solve velocity and pressure

VPmesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q2/dPc1"), 
                                   elementRes  = (int(res * aspect_ratio), res), 
                                   minCoord    = (0., 0.), 
                                   maxCoord    = (boxHeight*aspect_ratio, boxHeight))

Tmesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q2"), 
                                   elementRes  = (int(res * aspect_ratio), res), 
                                   minCoord    = (0., 0.), 
                                   maxCoord    = (boxHeight*aspect_ratio, boxHeight))

velocityField    = uw.mesh.MeshVariable( mesh=VPmesh,         nodeDofCount=2 ) # vector
pressureField    = uw.mesh.MeshVariable( mesh=VPmesh.subMesh, nodeDofCount=1 ) # scalar
temperatureField = uw.mesh.MeshVariable( mesh=Tmesh,          nodeDofCount=1 ) # scalar

velocityField.data[:,:] = 0.0
pressureField.data[:] = 0.0
temperatureField.data[:,0] = np.cos( Tmesh.data[:,0] * np.pi ) * np.sin( Tmesh.data[:,1] * np.pi )
# NOTE the 0 (try it without !)

In [ ]:
# Boundary conditions - free slip everywhere

vxWalls = VPmesh.specialSets["MinI_VertexSet"] + \
          VPmesh.specialSets["MaxI_VertexSet"] 

vyWalls = VPmesh.specialSets["MinJ_VertexSet"] + \
          VPmesh.specialSets["MaxJ_VertexSet"]

velocityBC = uw.conditions.DirichletCondition( variable = velocityField, 
                                               indexSetsPerDof = (vxWalls, vyWalls) )

In [ ]:
## Need a buoyancy function in the Stokes' solver

thermal_buoyancy = temperatureField * (0.0, 1.0)
 
stokesPIC = uw.systems.Stokes( velocityField  = velocityField, 
                               pressureField  = pressureField,
                               conditions     = [velocityBC,],
                               fn_viscosity   = 1.0, 
                               fn_bodyforce   = thermal_buoyancy )


solver = uw.systems.Solver( stokesPIC )

solver.solve()

In [ ]:
fig2 = glucifer.Figure()
fig2.append( glucifer.objects.VectorArrows( VPmesh, velocityField, arrowHead=0.35, scaling=3.0 ) )
fig2.append( glucifer.objects.Surface( VPmesh, temperatureField ) )
fig2.show()

## Things to try


Does the temperature mesh have to be the same element type / resolution as the velocity mesh ?

(If you try this in parallel, you might come to a different conclusion !)
